In [ ]:
from query.datasets.prelude import *
from query.datasets.ingest import ingest_pose

In [ ]:
hosts = pickle.load(open('/app/host_dict.pkl', 'rb'))

In [ ]:
hosts.keys()[0]

In [ ]:
Video.objects.all()[0].item_name()

In [ ]:
all_faces = []
videos = {v.item_name(): v for v in Video.objects.all()}
EPSILON = 0.0001
for path, bboxes in tqdm(list(hosts.iteritems())):
    v = videos[path]
    faces = collect(Face.objects.filter(person__frame__video=v, person__frame__number__in=[t[0] for t in bboxes]).select_related('person__frame'), 
                    lambda f: f.person.frame.number)
    for (frame, bbox) in bboxes:
        for f in faces[frame]:
            if abs(f.bbox_x1 - bbox['bbox_x1']) < EPSILON and abs(f.bbox_y2 - bbox['bbox_y2']) < EPSILON:
                f.is_host = True
                all_faces.append(f)
                break

In [ ]:
Face.objects.bulk_update(all_faces, update_fields=['is_host'], batch_size=100)

In [ ]:
Face.objects.filter(is_host=True).count()

In [ ]:
Video.objects.all()[0]

In [ ]:

vs = list(Video.objects.filter(time__year=2016, time__month=12, time__day=4).order_by('time').filter(channel__name='MSNBC'))
for v in vs:
    print(v.time.astimezone(pytz.timezone('America/Los_Angeles')), '{:.1f}'.format(v.num_frames/v.fps/3600))

In [ ]:
Video.objects.all()[0]

In [ ]:
videos = list(Video.objects.all())

In [ ]:
print(videos[0].path)
videos[0].time

In [ ]:
f = lambda x: \
    1          if x <= 1 else \
    1 + f(x-1) if x % 2 == 0 else \
    2 + f(x)-1

In [ ]:
#Speaker.objects.filter(labeler__name='handlabeled-audio').delete()
#Segment.objects.filter(labeler__name='handlabeled-audio:labeled').delete()

In [ ]:
tt={
    1: 'topic',
    2: 'phrase',
    3: 'person',
    4: 'location',
    5: 'organization'
}

     
for t in tqdm(list(Thing.objects.all())):
    t.type2 = ThingType.objects.get(name=tt[t.type])
    t.save()

In [ ]:
all_genders = list(FaceGender.objects.filter(labeler__name='handlabeled-gender').select_related('face', 'face__person', 'face__person__frame', 'face__person__frame__video'))

In [ ]:
FaceGender.objects.filter(labeler__name='rude-carnie').count()

In [ ]:
for_sahaj = {
    path: {
        frame: {k: [
            {
                'x1': f.face.bbox_x1,
                'x2': f.face.bbox_x2,
                'y1': f.face.bbox_y1,
                'y2': f.face.bbox_y2,
                'background': f.face.background,
                'gender': f.gender.name
            }
            for f in faces
        ] for k, faces in [('handlabeled', faces), ('mtcnn', FaceGender.objects.filter(labeler__name='rudecarnie', face__person__frame=faces[0].face.person.frame).select_related('face', 'face__person', 'face__person__frame', 'face__person__frame__video'))]}
        for frame, faces in collect(vid, lambda g: g.face.person.frame.number).iteritems()
    }
    for path, vid in collect(all_genders, lambda g: g.face.person.frame.video.path).iteritems()
}

In [ ]:
pickle.dump(for_sahaj, open('/app/sahaj-handlabeled-mtcnn.pkl', 'wb'))

In [ ]:
print(Frame.objects.filter(tags__name='handlabeled-face:labeled').count())
print(Frame.objects.filter(tags__name='tmp-verified').count())

# #Thing(name='don lemon', type=ThingType.PERSON).save()

In [ ]:
frames = Frame.objects.filter(tags__name='handlabeled-face:labeled')
t = 0
for f in frames:
    if Face.objects.filter(person__frame=f, labeler__name='handlabeled-face').count() == 0:
        print(f.id)
        t += 1
print(t)

In [ ]:
#pprint(list(Labeler.objects.values('name')))
Segment.objects.count()

In [ ]:
#tvnews%2Fvideos%2FMSNBCW_20170131_180000_MSNBC_Live.mp4&frame=102726
list(Face.objects.filter(person__frame__video__path='tvnews/videos/MSNBCW_20170131_180000_MSNBC_Live.mp4', person__frame__number=102726))

In [ ]:
Face.objects.filter(labeler__name='handlabeled-face').delete()
Frame.tags.through.objects.filter(tvnews_tag_id=Tag.objects.get(name='handlabeled-face:labeled')).delete()

In [ ]:
%matplotlib inline
import random
p = sns.color_palette("hls", 15).as_hex()
random.shuffle(p)
print(json.dumps(p))

In [ ]:
pprint(list(Video.objects.all().annotate( 
            hour=Extract('time', 'hour'), duration=Cast(F('num_frames') / F('fps'), models.IntegerField())) \
        .filter(hour=3).values('path', 'hour'))[:100])

In [ ]:
t = Tag.objects.get(name='handlabeled-face:labeled')
l = Labeler.objects.get(name='rudecarnie')
with Timer('a'):
    FaceGender.objects.filter(labeler=l).exclude(
                Q(face__shot__in_commercial=True)
                | Q(face__shot__video__commercials_labeled=False)
                | Q(face__shot__isnull=True)).order_by('?')[:10]

In [ ]:
spark = SparkWrapper()

In [ ]:
shots = spark.load('shots2', lambda: ())

In [ ]:
shots.where(shots.in_commercial == True).count()

In [ ]:
with Timer('test'):
    all_shots = list(Shot.objects.all())

In [ ]:
with Timer('test2'):
    all_shots2 = shots.select('id', 'in_commercial').collect()

In [ ]:
all_shots2 = {k.id: k.in_commercial for k in tqdm(all_shots2)}

In [ ]:
len(set(all_shots2.keys()) & set([s.id for s in all_shots]))

In [ ]:
for s in tqdm(all_shots):
    s.in_commercial = all_shots2[s.id]

In [ ]:
for i in tqdm(list(range(0, len(all_shots), 1000))):
    Shot.objects.bulk_update(all_shots[i:i+1000], batch_size=100, update_fields=['in_commercial'])

In [ ]:
print(Shot.objects.filter(in_commercial=False).count())
print(Shot.objects.filter(in_commercial=True).count())
print(len(all_shots))
print(Shot.objects.all().count())

In [ ]:
len([0 for s in tqdm(all_shots) if s.in_commercial])

In [ ]:
ids = list(Commercial.objects.distinct('video_id').values('video_id'))
videos = {v.id: v for v in Video.objects.all()}
for i in ids:
    videos[i['video_id']].commercials_labeled = True
print('b')
Video.objects.bulk_update(videos.values(), update_fields=['commercials_labeled'], batch_size=100)

In [ ]:
v = Video.objects.get(path__contains='MSNBCW_20171102_100000_Morning_Joe.mp4')
pprint(list(Commercial.objects.filter(video=v).values()))

In [ ]:
frame = Frame.objects.get(video=v, number=573126)
print(frame.id)
Face.objects.filter(person__frame=frame)[0].shot

In [ ]:
face = Face.objects.get(id=1101)
print(repr(face))
print(face.person.frame, face.person.frame.video.path)
#Face.objects.filter(person__frame__id=1130).count()

In [ ]:
video_map = {v.path: v for v in Video.objects.all()}
with open('/app/all_videos_dl.txt') as f:
    paths = ['tvnews/videos/{}.mp4'.format(s.strip()) for s in f.readlines()]
videos = [video_map[path] for path in paths]

In [ ]:
with Database() as db:
    tables = [db.table(v.path + '_blackframes') 
             for v in videos if db.has_table(v.path + '_blackframes') and db.table(v.path + '_blackframes').committed()]
    print(len(tables))
    
    def load(t):
        try:
            return pickle.loads(next(t.column('shots').load(rows=[t.num_rows() - 1]))[1])
        except Exception:
            traceback.print_exc()
            print(t.name())
            return None
    
    shots = par_for(load, tables, workers=8)
    print('Done!')

In [ ]:
pickle.dump([(t.name(), l) for t, l in zip(tables, shots)], open('all_shots.pkl', 'wb'), pickle.HIGHEST_PROTOCOL)

In [ ]:
as1 = pickle.load(open('all_shots.pkl', 'rb'))
as2 = pcache.get('all_shots_2')

In [ ]:
def bulk_load(db, videos):
    shots = [
        db.table(v.path + '_shots') for v in videos
        if db.has_table(v.path + '_shots') and db.table(v.path + '_shots').committed()
    ]

    def load(t):
        try:
            return pickle.loads(next(t.column('shots').load(rows=[t.num_rows() - 1]))[1])
        except Exception:
            traceback.print_exc()
            print(t.name())
            return None

    return par_for(load, shots, workers=8)

log.debug('Loading videos')
videos = list(Video.objects.annotate(
                c=Subquery(
                    Shot.objects.filter(video=OuterRef('pk')).values('video') \
                    .annotate(c=Count('video')).values('c')
                )).filter(c__isnull=False))
log.debug('Connecting to db')
with Database() as db:
    log.debug('Loading data')
    db._load_db_metadata()
    shots = bulk_load(db, videos)

In [ ]:
Frame.objects.filter(tags__name='openpose:labeled').count()

In [ ]:
pose = Pose.objects.get(id=1625727)
pose.pose_keypoints()[Pose.Nose]

In [ ]:
from query.datasets.tvnews.face_embed import face_embed

video = Video.objects.get(path='tvnews/videos/MSNBC_20100827_060000_The_Rachel_Maddow_Show.mp4')
face = Face.objects.get(id=119091)
FaceFeatures.objects.filter(face=face).delete()
face_embed([video], [[[face]]], force=True)


In [ ]:
face = Face.objects.get(id=119083)
FaceFeatures.objects.get(face=face).id

In [ ]:
db = Database()

In [ ]:
#Person.objects.filter(frame__video__path='tvnews/videos/MSNBC_20090703_010000_The_Rachel_Maddow_Show.mp4').delete()
#Face.objects.all().count()
#Video.objects.all().distinct('height').values('height')

with open('/app/paths') as f:
    paths = [s.strip() for s in f.readlines()]
    
tag, _ = Tag.objects.get_or_create(name='pose-test')    
for path in paths:
    video = Video.objects.get(path=path)
    VideoTag(video=video, tag=tag).save()

In [ ]:
Video.objects.filter(videotag__tag__name='pose-test').values('path')

In [ ]:
#FaceFeatures.objects.filter(face__person__frame__video__videotag__tag__name='pose-test').count()
#print Face.objects.all().order_by('facefeatures__distto').values('id').query
#PersonTrack.objects.filter(video__path='tvnews/videos/MSNBC_20100827_060000_The_Rachel_Maddow_Show.mp4') \
#    .annotate(duration=Track.duration()).values()

#id = 2043
#FaceFeatures.compute_distances(id)
tracks = list(PersonTrack.objects.filter(video__videotag__tag__name='pose-test') \
#tracks = list(PersonTrack.objects.filter(video__path='tvnews/videos/MSNBCW_20120303_020000_The_Rachel_Maddow_Show.mp4') \
    .annotate(c=Subquery(
        Face.objects.filter(person__tracks=OuterRef('pk')) \
        .filter(labeler__name='tinyfaces', facefeatures__distto__isnull=False, facefeatures__distto__lte=1.0) \
        .values('person__tracks')
        .annotate(c=Count('*'))
        .values('c'), models.IntegerField()
        )) \
    .filter(c__gt=0))
len(tracks)

In [ ]:
frames = defaultdict(set)    
for track in tracks:
    frames[track.video.path] |= set(range(track.min_frame, track.max_frame, 3))

# for path in frames.keys():    
#     if path == 'tvnews/videos/MSNBCW_20120303_020000_The_Rachel_Maddow_Show.mp4': continue
#     print(path)
#     #print('Deleting')
#     #Person.objects.filter(frame__video=Video.objects.get(path=path))
#     #Pose.objects.filter(person__frame__video=Video.objects.get(path=path)).delete()
#     print('Ingesting')
#     ingest_pose(
#         Video.objects.get(path=path),
#         db.table(path + '_poses_gather'),
#         sorted(frames[path]))
#     print('Ingested!')

with open('/app/chris-matthews-frames', 'w') as f:
    for path, frames in frames.iteritems():
        print(path)
        f.write('{} {}\n'.format(path, ' '.join([str(s) for s in sorted(frames)])))

In [ ]:
def closest_pose(candidates, target):
    noses = [pose.pose_keypoints()[Pose.Nose] for pose in candidates]
    noses, indices = zip(*[(nose[:2], i) for i, nose in enumerate(noses) if nose[2] > 0])
    dists = np.linalg.norm(np.array(noses) - target, axis=1)
    closest = candidates[indices[np.argmin(dists)]]
    return closest

In [ ]:
# Match detected poses with the sparse faces 
for track in tracks:
    faces = list(Face.objects.filter(person__tracks=track).select_related('person__frame'))
    for face in faces:
        poses = list(Pose.objects.filter(person__frame=face.person.frame))
        if len(poses) == 0:
            continue
        closest = closest_pose(poses, bbox_midpoint(face))
        closest.person = face.person
        closest.save()

In [ ]:
# Fill in tracks with poses
for track in tracks:
    faces = list(Face.objects.filter(person__tracks=track).select_related('person__frame') \
                 .order_by('person__frame__number'))
    for i in range(len(faces) - 1):
        try:
            last_pose = Pose.objects.get(person=faces[i].person)
            for j in range(faces[i].person.frame.number, faces[i+1].person.frame.number, 3):
                cur_poses = Pose.objects.filter(person__frame__video=track.video, person__frame__number=j)
                if len(cur_poses) == 0:
                    continue
                closest = closest_pose(cur_poses, last_pose.pose_keypoints()[Pose.Nose][:2])
                closest.person.tracks.add(track)
                closest.save()
                last_pose = closest
        except Pose.DoesNotExist:
            pass      

In [ ]:
for track in tracks:
    print(track.id, track.max_frame-track.min_frame, Person.objects.filter(tracks=track).count())

In [ ]:
# TODO: need to associate faces with poses
[p1, p2] = Pose.objects.all()[:2]

def pose_dist(p1, p2):
    kp1 = p1.pose_keypoints()
    kp2 = p2.pose_keypoints()
    
    weights = defaultdict(float, {
        Pose.LWrist: 0.4,
        Pose.RWrist: 0.4,
        Pose.Nose: 0.1,        
        Pose.LElbow: 0.05,
        Pose.RElbow: 0.05
    })
    weight_vector = [weights[i] for i in range(Pose.POSE_KEYPOINTS)]
    
    dist = np.linalg.norm(kp2[:,:2] - kp1[:,:2], axis=1)
    weighted_dist = np.array([d * w for d, s, w in zip(dist, kp2[:, 2], weight_vector) if s > 0])
    return np.linalg.norm(weighted_dist)

In [ ]:
all_dists = []
for track in tracks:
    poses = list(Pose.objects.filter(person__tracks=track).order_by('person__frame__number'))
    dists = [pose_dist(poses[i], poses[i+1]) for i in range(len(poses) - 1)]
    all_dists.append((track.id, np.mean(dists), track.duration()))

pprint(sorted(all_dists, key=itemgetter(1), reverse=True))